# Mission to Mars
Rutgers Data Science Bootcamp Week 13 Assignement

## Python Imports

In [9]:
from bs4 import BeautifulSoup as bs4
import pandas as pd
import requests
from splinter import Browser
from pymongo import MongoClient
from flask import Flask

URL = 'https://mars.nasa.gov/news/'
client = MongoClient('localhost', 27017)

In [ ]:
with Browser('Chrome') as browser:
    browser.visit(URL)